In our work we decided to use the data from the 2024, 2023 and 2022 F1 championships to predict which racers would finish on which places during the Grand Prix Emilia-Romani race, which will take place on May 19. In total we collected 31 records from revious races and trained 4 Machine Learning models: Linear Regression, AutoRegressive Integrated Moving Average, Simple Exponential Smoothing and Holt’s Linear Smoothing with different hyperparameters. We also used the Miami Grand Prix race (the 31th record) to test our forecasting performance and based on the results chose the best model. 

Unfortunately, we were unable to gather a sufficent amount of data for all drivers from the previous competitions (2021, 2020 etc.) because many drivers simply did not take part in competitions ogranized a couple of years ago. Moreover, since we try to predict a future event based on data from previous races, it's more a time-series forecasting method where splitting the data into training and test sets is not necessary as in other ML tasks. Therefore, we used the earlier portion of data (30 races) for training and the later portion (race in Miami) for testing the predictions.


After determining, that the Linear Regression model gave us best predictions, we once again trained the LR model to include the data from the last race from Miami and forecasted the possible top 5 racers in the Emilia-Romani race which could be seen at the very bottom. We also sending you our excel file with the data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In the first place, we would like to import the file with the data from the last 31 races

In [5]:
df = pd.read_excel(r"DSwP_final.xlsx")
df.set_index('Driver', inplace=True)
df_with_miami = df
df = df.drop('MIA24',axis = 1)
df.head(5)

,MXC22,SAP22,ABU22,BHR23,SAU23,AUS23,AZE23,MIA23,MON23,ESP23,...,USA23,MXC23,SAP23,LVG23,ABU23,BHR24,SAU24,AUS24,JPN24,CHN24
Driver,,,,,,,,,,,,,,,,,,,,,
Max Verstappen,1.0,6.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
Sergio Pérez,3.0,7.0,3.0,2.0,1.0,5.0,1.0,2.0,16.0,4.0,...,4.0,NaN,4.0,3.0,4.0,2.0,2.0,5.0,2.0,3.0
Charles Leclerc,6.0,4.0,2.0,NaN,7.0,NaN,3.0,7.0,6.0,11.0,...,NaN,3.0,NaN,2.0,2.0,4.0,3.0,2.0,4.0,4.0
Carlos Sainz Jr.,5.0,3.0,4.0,4.0,6.0,12.0,5.0,5.0,8.0,5.0,...,3.0,4.0,6.0,6.0,18.0,3.0,NaN,1.0,3.0,5.0
Lando Norris,9.0,NaN,6.0,17.0,17.0,6.0,9.0,17.0,9.0,17.0,...,2.0,5.0,2.0,NaN,5.0,6.0,8.0,3.0,5.0,2.0


We used NA to indicate occurences when the racer did not take part in the race, has not finished it due to various reasons etc. We will get rid of missing data in the following parts of our report

Let's get some statistics about our drivers:

In [6]:
df.transpose().describe()

Driver,Max Verstappen,Sergio Pérez,Charles Leclerc,Carlos Sainz Jr.,Lando Norris,Oscar Piastri,George Russell,Fernando Alonso,Lewis Hamilton,Lance Stroll,...,Oliver Bearman,Nico Hülkenberg,Kevin Magnussen,Alexander Albon,Esteban Ocon,Zhou Guanyu,Daniel Ricciardo,Pierre Gasly,Valtteri Bottas,Logan Sargeant
count,29.000000,28.000000,25.000000,27.000000,28.000000,24.000000,27.000000,27.000000,27.000000,25.000000,...,1.0,26.000000,26.000000,25.000000,24.000000,26.000000,12.000000,28.000000,26.000000,22.000000
mean,1.379310,4.178571,4.480000,5.555556,6.964286,8.958333,6.555556,6.333333,5.703704,9.760000,...,7.0,13.692308,14.769231,11.560000,10.166667,13.461538,12.166667,11.214286,13.538462,15.500000
std,1.177582,3.139398,2.311565,3.467801,5.022041,4.368655,3.974276,3.902662,3.450918,3.643259,...,NaN,2.949837,2.997435,2.550817,3.667325,2.387145,3.128559,3.593792,3.478284,2.939874
min,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,4.000000,...,7.0,7.000000,10.000000,7.000000,3.000000,9.000000,7.000000,3.000000,8.000000,10.000000
25%,1.000000,2.000000,3.000000,3.500000,2.750000,5.750000,4.500000,3.000000,3.000000,7.000000,...,7.0,12.000000,13.000000,10.000000,8.000000,12.000000,10.500000,9.000000,11.000000,13.250000
50%,1.000000,3.000000,4.000000,5.000000,6.000000,8.000000,6.000000,6.000000,5.000000,10.000000,...,7.0,14.000000,15.000000,12.000000,9.500000,13.000000,13.000000,11.000000,13.500000,16.000000
75%,1.000000,5.000000,6.000000,6.000000,9.000000,11.250000,7.000000,8.000000,7.500000,12.000000,...,7.0,15.750000,17.000000,14.000000,13.250000,15.000000,14.250000,13.250000,16.500000,17.000000
max,6.000000,16.000000,11.000000,18.000000,17.000000,19.000000,17.000000,19.000000,18.000000,17.000000,...,7.0,19.000000,20.000000,16.000000,17.000000,18.000000,16.000000,18.000000,19.000000,20.000000


We've decided to replace NA values in the dataset with the mean of the places in all races for each driver using the replace_na_with_row_mean function below. We've also decided to exclude Oliver Bearman from our race statistics, because we have only 1 observation for him, which may heavily skew the results (He is probably just a spare driver who participated only once)

In [7]:
df = df.drop('Oliver Bearman')

In [8]:
def replace_na_with_row_mean(df):
    row_means = df.mean(axis=1)
    for index, row in df.iterrows():
        nan_indices = row.index[row.isna()]
        if len(nan_indices) > 0:
            df.loc[index, nan_indices] = row_means[index]
    return df

In [9]:
replace_na_with_row_mean(df_with_miami)
replace_na_with_row_mean(df)
df.isna().sum()

MXC22    0
SAP22    0
ABU22    0
BHR23    0
SAU23    0
AUS23    0
AZE23    0
MIA23    0
MON23    0
ESP23    0
CAN23    0
AUT23    0
GBR23    0
HUN23    0
BEL23    0
NED23    0
ITA23    0
SIN23    0
JPN23    0
QAT23    0
USA23    0
MXC23    0
SAP23    0
LVG23    0
ABU23    0
BHR24    0
SAU24    0
AUS24    0
JPN24    0
CHN24    0
dtype: int64

There are no NA values in our dataset anymore, so we are good to go to the next step which is training ML models. First of all, we transformed the column names to numeric values in order to use it in the prediction of final places later on.

In [10]:
df.columns = range(1,31)
df.head(5)

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
Driver,,,,,,,,,,,,,,,,,,,,,
Max Verstappen,1.0,6.000000,1.0,1.00,2.0,1.00,2.0,1.0,1.0,1.0,...,1.00,1.000000,1.00,1.000000,1.0,1.0,1.000000,1.37931,1.0,1.0
Sergio Pérez,3.0,7.000000,3.0,2.00,1.0,5.00,1.0,2.0,16.0,4.0,...,4.00,4.178571,4.00,3.000000,4.0,2.0,2.000000,5.00000,2.0,3.0
Charles Leclerc,6.0,4.000000,2.0,4.48,7.0,4.48,3.0,7.0,6.0,11.0,...,4.48,3.000000,4.48,2.000000,2.0,4.0,3.000000,2.00000,4.0,4.0
Carlos Sainz Jr.,5.0,3.000000,4.0,4.00,6.0,12.00,5.0,5.0,8.0,5.0,...,3.00,4.000000,6.00,6.000000,18.0,3.0,5.555556,1.00000,3.0,5.0
Lando Norris,9.0,6.964286,6.0,17.00,17.0,6.00,9.0,17.0,9.0,17.0,...,2.00,5.000000,2.00,6.964286,5.0,6.0,8.000000,3.00000,5.0,2.0


The first model we would like to train is a simple linear regression. Linear regression assumes a linear relationship between the independent variable(s) and the dependent variable, which might not always hold true in time series data where patterns can be more complex. However, linear regression can still be useful in certain cases, especially if the relationship between the variables appears to be linear and relatively stable over time.

We will use a function called LR_pred to construct the regression for each of the driver's places in last 30 races

In [11]:
def LR_pred(df):
    preds = []
    for x in range(0,len(df)):
        model = LinearRegression()
        model.fit(pd.DataFrame(df.columns[1:]), list(df.iloc[x])[1:])
        prediction = model.predict(np.array([[29]]).reshape(-1, 1))
        preds.append(prediction[0])
    return preds

In [12]:
LR_predictions = LR_pred(df)

The second model that we would like to train is called ARIMA which stands for AutoRegressive Integrated Moving Average. It's a class of models that captures a suite of different standard temporal structures in time series data. But before we use ARIMA, we need to ensure that our data is stationary, which involves checking if its statistical properties such as mean, variance, and autocorrelation are constant over time.

We will  use the Augmented Dickey-Fuller (ADF) test to check for stationarity

In [13]:
from statsmodels.tsa.stattools import adfuller

In [14]:
for x in range(0,len(df)):
    adf_test = adfuller(df.iloc[x])
    print(df.index[x])
    print('ADF Statistic: %f' % adf_test[0])
    print('p-value: %f' % adf_test[1])

Max Verstappen
ADF Statistic: -5.835015
p-value: 0.000000
Sergio Pérez
ADF Statistic: -3.430977
p-value: 0.009947
Charles Leclerc
ADF Statistic: -0.738716
p-value: 0.836465
Carlos Sainz Jr.
ADF Statistic: -4.901721
p-value: 0.000035
Lando Norris
ADF Statistic: -2.898384
p-value: 0.045536
Oscar Piastri
ADF Statistic: -3.944774
p-value: 0.001729
George Russell
ADF Statistic: -5.259494
p-value: 0.000007
Fernando Alonso
ADF Statistic: -6.012064
p-value: 0.000000
Lewis Hamilton
ADF Statistic: -0.899661
p-value: 0.788084
Lance Stroll
ADF Statistic: -1.476308
p-value: 0.545197
Yuki Tsunoda
ADF Statistic: -5.636908
p-value: 0.000001
Nico Hülkenberg
ADF Statistic: -4.613845
p-value: 0.000122
Kevin Magnussen
ADF Statistic: -5.280092
p-value: 0.000006
Alexander Albon
ADF Statistic: -4.338700
p-value: 0.000380
Esteban Ocon
ADF Statistic: -3.594675
p-value: 0.005864
Zhou Guanyu
ADF Statistic: -2.264023
p-value: 0.183834
Daniel Ricciardo
ADF Statistic: -4.150766
p-value: 0.000797
Pierre Gasly
ADF St

A p-value below 0.05 indicates stationarity, and the majority of our data meets this criterion. Unfortunately, for drivers such as Charles Leclerc, Lewis Hamilton, Lance Stroll and Zhou Guanyu the test has shown unstationarity, so we will exclude them from the ML training now and just impute their values with the mean of their races in the 30 races in the final statistic in order not to loose data about those drivers altogether.

In [15]:
df_for_ML = df.drop(['Charles Leclerc','Lewis Hamilton', 'Lance Stroll', 'Zhou Guanyu'])

In [16]:
Driver_means = df.loc[['Charles Leclerc','Lewis Hamilton', 'Lance Stroll', 'Zhou Guanyu']].mean(axis=1)

We will use a function called ARIMA_pred to construct the ARIMA model for each driver's observations

In [17]:
from statsmodels.tsa.arima.model import ARIMA

In [18]:
def ARIMA_pred(df):
    preds = []
    for x in range(0,len(df)):
        model = ARIMA(df.iloc[x])
        model_fit = model.fit()
        preds.append(model_fit.forecast(steps = 1)[31])
    return preds

In [19]:
df_full_AIRMA = pd.concat([pd.DataFrame(ARIMA_pred(df_for_ML), df_for_ML.index), 
                           pd.DataFrame(Driver_means)])

The next ML method we focused on is Exponential Smoothing (ETS). ETS models are another class of time series forecasting methods that can capture trend and seasonality in the data. They're useful when you have a relatively small dataset, as in our case. The technique assigns larger weights to more recent observations while assigning exponentially decreasing weights as the observations get increasingly distant. We will use Simple Exponential Smoothing and Double Exponential Smoothing (Holt's method)

In ETS and DES the mean and standard deviation also need to be stationary. We checked it before training our last ML model, and only 16 out of 20 are stationary, so we will use ETS to predict them. For other 4 variables we will simply use the mean of races, as in the previous model. We will also use this ML algorithm with different values of its hyperparameter (smoothing_level) in order to choose the best version of the model.

In [20]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [21]:
def ETS_pred(df):
    preds = []
    preds_2 = []
    preds_5 = []
    for x in range(0,len(df)):
        model = SimpleExpSmoothing(df.iloc[x])
        fit1 = model.fit()
        preds.append(fit1.forecast(steps = 1)[31])
        fit2 = model.fit(smoothing_level=.2)
        preds_2.append(fit2.forecast(steps = 1)[31])
        fit3 = model.fit(smoothing_level=.5)
        preds_5.append(fit3.forecast(steps = 1)[31])
    return preds, preds_2, preds_5

In [22]:
data_ETS = {
    'ETS_pred_for_def': ETS_pred(df_for_ML)[0],
    'ETS_pred_for_0.2': ETS_pred(df_for_ML)[1],
    'ETS_pred_for_0.5': ETS_pred(df_for_ML)[2],
}
data_ETS_means = {
    'ETS_pred_for_def': Driver_means,
    'ETS_pred_for_0.2': Driver_means,
    'ETS_pred_for_0.5': Driver_means,
}

In [23]:
df_full_ETS = pd.concat([pd.DataFrame(data_ETS, df_for_ML.index), 
                           pd.DataFrame(data_ETS_means)])

In [24]:
df_full_ETS

,ETS_pred_for_def,ETS_pred_for_0.2,ETS_pred_for_0.5
Driver,,,
Max Verstappen,1.379310,1.108577,1.047902
Sergio Pérez,4.178572,3.362737,2.933348
Carlos Sainz Jr.,5.555555,4.930491,4.183118
Lando Norris,4.118214,4.430874,3.467627
Oscar Piastri,7.432215,7.207728,7.254750
George Russell,6.555556,7.574525,7.564453
Fernando Alonso,6.333333,6.858286,6.811223
Yuki Tsunoda,12.500000,11.586143,11.345421
Nico Hülkenberg,13.692308,12.038552,10.480528


The last model we would like to train is Holt’s Linear Smoothing. It's an extended simple exponential smoothing algorithm that allows the forecasting of data with a trend. This method involves a forecast equation and two smoothing equations (one for the level and one for the trend). Holt's Linear Trend Method is also called Double Exponential Smoothing. As well as in the previous models, the data for this model must be stationary.

In [25]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt

In [26]:
def ETS_pred(df):
    preds_tuned = []
    preds_optimized = []
    for x in range(0,len(df)):
        model = Holt(np.asarray(df.iloc[x]))
        fit1 = model.fit(smoothing_level=.3, smoothing_trend=.1)
        preds_tuned.append(fit1.forecast(steps = 1)[0])
        fit2 = model.fit(optimized=True)
        preds_optimized.append(fit2.forecast(steps = 1)[0])
    return preds_tuned, preds_optimized

In [27]:
data_H = {
    'Holt_pred_tuned': ETS_pred(df_for_ML)[0],
    'Holt_pred_optimized': ETS_pred(df_for_ML)[1],
}
data_H_means = {
    'Holt_pred_tuned': Driver_means,
    'Holt_pred_optimized': Driver_means,
}

In [28]:
df_full_Holt = pd.concat([pd.DataFrame(data_H, df_for_ML.index), 
                           pd.DataFrame(data_H_means)])

Now we would like to present you the table of predictions for all models on the last race, which took place in Miami on the 5th of May.

In [29]:
full_pred_data = {
    'Average': df.mean(axis=1).sort_values().index,
    'Linear_regression': pd.DataFrame(df.index, LR_predictions).sort_index()['Driver'].values,
    'AIRMA_pred': df_full_AIRMA.sort_values(by=0).index,
    'ETS_pred_for_def': df_full_ETS['ETS_pred_for_def'].sort_values().index,
    'ETS_pred_for_0.2': df_full_ETS['ETS_pred_for_0.2'].sort_values().index,
    'ETS_pred_for_0.5': df_full_ETS['ETS_pred_for_0.5'].sort_values().index,
    'Holt_pred_tuned': df_full_Holt['Holt_pred_tuned'].sort_values().index,
    'Holt_pred_optimized': df_full_Holt['Holt_pred_optimized'].sort_values().index
}

In [30]:
predictions_for_miami = pd.DataFrame(full_pred_data, index = range(1,21)).head(5)

In [31]:
print(predictions_for_miami)

            Average Linear_regression        AIRMA_pred  ETS_pred_for_def  \
1    Max Verstappen    Max Verstappen    Max Verstappen    Max Verstappen   
2      Sergio Pérez      Lando Norris      Sergio Pérez      Lando Norris   
3   Charles Leclerc   Charles Leclerc   Charles Leclerc      Sergio Pérez   
4  Carlos Sainz Jr.      Sergio Pérez  Carlos Sainz Jr.   Charles Leclerc   
5    Lewis Hamilton  Carlos Sainz Jr.    Lewis Hamilton  Carlos Sainz Jr.   

   ETS_pred_for_0.2  ETS_pred_for_0.5   Holt_pred_tuned Holt_pred_optimized  
1    Max Verstappen    Max Verstappen    Max Verstappen      Max Verstappen  
2      Sergio Pérez      Sergio Pérez      Sergio Pérez        Lando Norris  
3      Lando Norris      Lando Norris      Lando Norris        Sergio Pérez  
4   Charles Leclerc  Carlos Sainz Jr.  Carlos Sainz Jr.     Charles Leclerc  
5  Carlos Sainz Jr.   Charles Leclerc   Charles Leclerc    Carlos Sainz Jr.  


As you can see from the data above, the forecasts for our models can be really different. 
Now let's compare our predistions with the actual places of the drivers in the Miami race

In [32]:
df_with_miami['MIA24'].sort_values()[:5].index

Index(['Lando Norris', 'Max Verstappen', 'Charles Leclerc', 'Sergio Pérez',
       'Carlos Sainz Jr.'],
      dtype='object', name='Driver')

In [33]:
top_places = df_with_miami['MIA24'].sort_values()[:5].index
top_places

Index(['Lando Norris', 'Max Verstappen', 'Charles Leclerc', 'Sergio Pérez',
       'Carlos Sainz Jr.'],
      dtype='object', name='Driver')

Now let's check how many top-5 places have been succesfully forecasted by each of our trained ML models

In [34]:
def check_places(df,top_5_places):
    true_preds = {}
    for column in df.columns:
        correct = 0
        for x in range(0,5):
            if top_5_places[x] == df[column].iloc[x]:
                correct += 1
        true_preds[column] = correct
    return true_preds

In [35]:
check_places(predictions_for_miami, top_places)

{'Average': 1,
 'Linear_regression': 3,
 'AIRMA_pred': 1,
 'ETS_pred_for_def': 1,
 'ETS_pred_for_0.2': 1,
 'ETS_pred_for_0.5': 0,
 'Holt_pred_tuned': 0,
 'Holt_pred_optimized': 1}

In [36]:
def check_racers(df,top_5_places):
    true_preds = {}
    for column in df.columns:
        correct = 0
        for x in range(0,5):
            if top_5_places[x] in list(df[column]):
                correct += 1
        true_preds[column] = correct
    return true_preds

In [37]:
check_racers(predictions_for_miami, top_places)

{'Average': 4,
 'Linear_regression': 5,
 'AIRMA_pred': 4,
 'ETS_pred_for_def': 5,
 'ETS_pred_for_0.2': 5,
 'ETS_pred_for_0.5': 5,
 'Holt_pred_tuned': 5,
 'Holt_pred_optimized': 5}

As we can see from the dictionary above, the linear regression model was the most successfull when predicting the exact place for each racer in the first top 5 places in the Miami race (3 out of 5). Moreover, it also correctly identified all the top racers regardless of their place (5 out of 5). 

From this example we can clearly see, that sometimes more sophisticated models are not needed, it's enough to train a more popular or easy model to achieve good results :)

In the end, we will once again train a LR model on a full dataset, including the Miami race, and generate predictions for the Formula 1 Grand Prix Emilia-Romani race, which will take place on May 19th.

In [38]:
def LR_pred_full(df):
    preds = []
    for x in range(0,len(df)):
        model = LinearRegression()
        model.fit(pd.DataFrame(df.columns[1:]), list(df.iloc[x])[1:])
        prediction = model.predict(np.array([[30]]).reshape(-1, 1))
        preds.append(prediction[0])
    return preds

In [39]:
LR_predict_final = LR_pred_full(df)

In [40]:
pd.DataFrame(df.index, LR_predict_final).sort_index().head(5)

,Driver
0.858264,Max Verstappen
2.447291,Lando Norris
3.284138,Charles Leclerc
3.685468,Sergio Pérez
5.221456,Carlos Sainz Jr.


In [41]:
pd.DataFrame(df.index, LR_predict_final).sort_index()['Driver'].values[0:5]

array(['Max Verstappen', 'Lando Norris', 'Charles Leclerc',
       'Sergio Pérez', 'Carlos Sainz Jr.'], dtype=object)